# **STEP 1: Mount Drive and Setup Paths**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = "/content/drive/MyDrive/octdataset"
os.chdir(project_path)
print("Current directory:", os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current directory: /content/drive/MyDrive/octdataset


# **STEP 2: Import Libraries**

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from lightgbm import LGBMClassifier
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import seaborn as sns
from google.colab import files

# **STEP 3: Load OCT + Fundus Image Data**

In [ ]:
oct_folder = "/content/drive/MyDrive/octdataset/OCT/OCT_NEW"
fundus_folder = "/content/drive/MyDrive/octdataset/eye fundus (2)/eye fundus"

In [ ]:
def load_images_from_folder(folder, label, img_size=128, limit=None):
    images, labels = [], []
    count = 0
    for root, _, files in os.walk(folder):
        for img_name in files:
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.bmp')):
                img_path = os.path.join(root, img_name)
                try:
                    img = cv2.imread(img_path)
                    if img is None:
                        continue
                    img = cv2.resize(img, (img_size, img_size))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    images.append(img)
                    labels.append(label)
                    count += 1
                    if limit and count >= limit:
                        return np.array(images), np.array(labels)
                except Exception as e:
                    print("Error loading:", img_path, e)
                    continue
    return np.array(images), np.array(labels)

In [ ]:
# Load OCT (label 0) and Fundus (label 1)
oct_images, oct_labels = load_images_from_folder(oct_folder, label=0)
fundus_images, fundus_labels = load_images_from_folder(fundus_folder, label=1)

In [ ]:
# Combine images and labels
images = np.concatenate((oct_images, fundus_images), axis=0) / 255.0
labels = np.concatenate((oct_labels, fundus_labels), axis=0)

print(f"Total images: {images.shape}, Total labels: {labels.shape}")

# **STEP 4: Feature Extraction using VGG16**

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

print("Extracting deep features...")
features = model.predict(images, verbose=1)
features = features.reshape(features.shape[0], -1)
print("Feature shape:", features.shape)

# **STEP 5: Train-Test Split (80% train, 20% test)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# **STEP 6: Train LightGBM**

In [ ]:
lgbm = LGBMClassifier(
    n_estimators=150,
    learning_rate=0.05,
    max_depth=10,
    num_leaves=31,
    random_state=42,
    force_col_wise=True,
    verbosity=-1
)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)
lgbm_acc = accuracy_score(y_test, lgbm_pred)
print(f"\n✅ LightGBM Accuracy: {lgbm_acc*100:.2f}%")

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, lgbm_pred))

cm = confusion_matrix(y_test, lgbm_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - LightGBM")
plt.show()

# **STEP 7: Upload and Predict User Image**

In [ ]:
# STEP 8: Upload and Predict User Image
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

test_img = cv2.imread(img_path)
test_img = cv2.resize(test_img, (128, 128))
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_img = test_img / 255.0
test_img = np.expand_dims(test_img, axis=0)

test_features = model.predict(test_img)
test_features = test_features.reshape(test_features.shape[0], -1)
test_features = scaler.transform(test_features)

loaded_model = joblib.load("best_dr_model.pkl")
pred = loaded_model.predict(test_features)[0]
proba = loaded_model.predict_proba(test_features)[0]
confidence = np.max(proba) * 100

label_map = {0: "Normal", 1: "Moderate", 2: "Severe"}
print(f"\n🩺 Prediction: {label_map.get(pred, 'Unknown')} (Confidence: {confidence:.2f}%)")

# **STEP 8:Images Before and After Preprocessing**

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
# Select random samples from OCT and Fundus datasets
def show_sample_images(before_images, after_images, title_before, title_after):
    plt.figure(figsize=(10, 5))
    for i in range(3):
        # Before Preprocessing
        plt.subplot(2, 3, i+1)
        plt.imshow(before_images[i])
        plt.title(f"{title_before} {i+1}")
        plt.axis('off')

        # After Preprocessing
        plt.subplot(2, 3, i+4)
        plt.imshow(after_images[i])
        plt.title(f"{title_after} {i+1}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Load a few raw images before preprocessing
sample_oct_raw = [cv2.imread(os.path.join(oct_folder, img)) for img in os.listdir(oct_folder)[:3]]
sample_fundus_raw = [cv2.imread(os.path.join(fundus_folder, img)) for img in os.listdir(fundus_folder)[:3]]

In [ ]:
# Apply preprocessing to the same images
def preprocess_images(image_list, img_size=128):
    processed = []
    for img in image_list:
        if img is None:
            continue
        img = cv2.resize(img, (img_size, img_size))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        processed.append(img)
    return np.array(processed)

In [ ]:
sample_oct_processed = preprocess_images(sample_oct_raw)
sample_fundus_processed = preprocess_images(sample_fundus_raw)

In [ ]:
# Display before and after preprocessing
print("📷 OCT Images Before and After Preprocessing:")
show_sample_images(sample_oct_raw, sample_oct_processed, "Before", "After")

In [ ]:
print("📷 Fundus Images Before and After Preprocessing:")
show_sample_images(sample_fundus_raw, sample_fundus_processed, "Before", "After")